# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
!pip install cartopy geoviews pyproj


In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Convert to Celsius 
city_data_df['Max Temp'] = (city_data_df['Max Temp'] - 32) * 5/9
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,12.900000,97,100,4.63,NZ,1707062669
1,1,port-aux-francais,-49.3500,70.2167,4.688889,81,77,27.67,TF,1707062669
2,2,menongue,-14.6585,17.6910,20.927778,89,100,5.53,AO,1707062669
3,3,saudarkrokur,65.7461,-19.6394,-11.500000,85,90,4.94,IS,1707062555
4,4,college,64.8569,-147.8028,-30.311111,70,20,0.00,US,1707062488


In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# %%capture --no-display

# Ensure data has a correct datatype
city_data_df['Lat'] = city_data_df['Lat'].astype(float)
city_data_df['Lng'] = city_data_df['Lng'].astype(float)
city_data_df['Humidity'] = city_data_df['Humidity'].astype(float)

# Configure the map plot
plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', 
                                  size='Humidity', color='Humidity', 
                                  hover_cols=['City', 'Country', 'Humidity'], 
                                  scale=0.1, alpha=0.6, frame_width=700, frame_height=500)


# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Let's assume a crierias to narrow down our data related to this:
# * Maximum temperature between 20°C to 28°C (approximately 70°F and 80°F)
# * Wind speed less than 20 km/h
# * Zero cloudiness
# * Humidity between 40% and 65%

# Define ideal weather conditions in Celsius
ideal_temp = (city_data_df['Max Temp'] > 20) & (city_data_df['Max Temp'] < 28)
ideal_wind = (city_data_df['Wind Speed'] < 20)
ideal_cloudiness = (city_data_df['Cloudiness'] == 0)
ideal_humidity = (city_data_df['Humidity'] > 40) & (city_data_df['Humidity'] < 65)


ideal_conditions = ideal_temp & ideal_wind & ideal_cloudiness & ideal_humidity


# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[ideal_conditions].copy()


# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,la mira,18.0350,-102.3266,25.322222,57.0,0,1.21,MX,1707062675
122,122,ciudad lazaro cardenas,17.9583,-102.2000,24.961111,59.0,0,0.60,MX,1707062684
200,200,talcahuano,-36.7167,-73.1167,21.822222,56.0,0,6.91,CL,1707062411
224,224,port isabel,26.0734,-97.2086,23.022222,51.0,0,4.61,US,1707062696
229,229,cabo san lucas,22.8909,-109.9124,21.527778,54.0,0,1.01,MX,1707062600


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""


# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
48,la mira,MX,18.0350,-102.3266,57.0,
122,ciudad lazaro cardenas,MX,17.9583,-102.2000,59.0,
200,talcahuano,CL,-36.7167,-73.1167,56.0,
224,port isabel,US,26.0734,-97.2086,51.0,
229,cabo san lucas,MX,22.8909,-109.9124,54.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Double check the API keys import
from api_keys import geoapify_key  


# Set parameters to search for a hotel

# radius 10KM 
radius = 10000 


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']

    params = {
        'categories': 'accommodation.hotel',
        'filter': f'circle:{lng},{lat},{radius}',
        'limit': 1,  # We only want the first hotel
        'apiKey': geoapify_key
    }

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # To ensure HTTP request errors are captured
        name_address = response.json()
        
        # Grab the first hotel from the results and store the name
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
la mira - nearest hotel: Villa Dorada
ciudad lazaro cardenas - nearest hotel: Quinta Antigua Hotel
talcahuano - nearest hotel: Wyndham Concepción Pettra
port isabel - nearest hotel: Alta Vista Apartments
cabo san lucas - nearest hotel: Hotel Riu Santa Fe
pathein - nearest hotel: Day To Day Motel
iquique - nearest hotel: Hotel Terrado Suites
sittwe - nearest hotel: Noble
rio gallegos - nearest hotel: Avellaneda Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
48,la mira,MX,18.0350,-102.3266,57.0,Villa Dorada
122,ciudad lazaro cardenas,MX,17.9583,-102.2000,59.0,Quinta Antigua Hotel
200,talcahuano,CL,-36.7167,-73.1167,56.0,Wyndham Concepción Pettra
224,port isabel,US,26.0734,-97.2086,51.0,Alta Vista Apartments
229,cabo san lucas,MX,22.8909,-109.9124,54.0,Hotel Riu Santa Fe


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:

# Configure the map plot
plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM',
                              size='Humidity',  # Size by humidity
                              hover_cols=['City', 'Hotel Name', 'Country', 'Humidity'],
                              color='Humidity',  # Color by humidity
                              scale=1, alpha=0.6, frame_width=700, frame_height=500,
                              title="Map of Cities with Ideal Weather and Hotels")


# Display the map
plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)